## Deliverable 2. Create a Customer Travel Destinations Map.

In [34]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [35]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kaitangata,NZ,-46.28,169.85,45.00,68,82,1.99,broken clouds
1,1,Dikson,RU,73.51,80.55,42.19,93,20,12.64,few clouds
2,2,Vaini,TO,-21.20,-175.20,72.09,92,100,27.13,moderate rain
3,3,Punta Arenas,CL,-53.15,-70.92,46.00,65,20,16.11,few clouds
4,4,San Patricio,MX,19.22,-104.70,85.41,64,67,2.89,broken clouds


In [36]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [37]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,San Patricio,MX,19.22,-104.70,85.41,64,67,2.89,broken clouds
13,13,Auki,SB,-8.77,160.70,76.06,85,100,6.67,overcast clouds
14,14,Butaritari,KI,3.07,172.79,82.74,74,26,13.76,scattered clouds
17,17,Cayenne,GF,4.93,-52.33,87.80,62,16,6.93,few clouds
22,22,Jamestown,US,42.10,-79.24,78.01,83,1,3.36,clear sky
23,23,Caxito,AO,-8.58,13.66,77.00,73,20,9.17,few clouds
31,31,Torbay,CA,47.67,-52.73,77.00,69,75,10.29,broken clouds
34,34,Hithadhoo,MV,-0.60,73.08,82.67,77,63,12.48,broken clouds
37,37,Bengkulu,ID,-3.80,102.27,76.55,90,100,6.29,light rain
41,41,Beeville,US,28.40,-97.75,87.01,94,40,5.82,scattered clouds


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                272
City                   272
Country                269
Lat                    272
Lng                    272
Max Temp               272
Humidity               272
Cloudiness             272
Wind Speed             272
Current Description    272
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [42]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,San Patricio,MX,85.41,broken clouds,19.22,-104.70,
13,Auki,SB,76.06,overcast clouds,-8.77,160.70,
14,Butaritari,KI,82.74,scattered clouds,3.07,172.79,
17,Cayenne,GF,87.80,few clouds,4.93,-52.33,
22,Jamestown,US,78.01,clear sky,42.10,-79.24,
23,Caxito,AO,77.00,few clouds,-8.58,13.66,
31,Torbay,CA,77.00,broken clouds,47.67,-52.73,
34,Hithadhoo,MV,82.67,broken clouds,-0.60,73.08,
37,Bengkulu,ID,76.55,light rain,-3.80,102.27,
41,Beeville,US,87.01,scattered clouds,28.40,-97.75,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])

In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))